<a href="https://colab.research.google.com/github/abxda/python/blob/main/UP_Semana_4_Viernes_y_S%C3%A1bado_01_Nacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
from zipfile import ZipFile
import requests
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from math import sin, cos, radians, sqrt
import matplotlib.pyplot as plt
import folium

In [ ]:
def download(url, directory):
    filename = url.split('/')[-1]
    filepath = os.path.join(directory, filename)

    # Verificar si el archivo ya existe
    if os.path.exists(filepath):
        print(f"El archivo {filename} ya existe, no se descarga de nuevo.")
        return

    # Proceso de descarga
    time.sleep(5)  # Simular retardo en la red
    r = requests.get(url, stream=True)
    total_size = int(r.headers['content-length'])
    with open(filepath, 'wb') as f:
        for data in tqdm(iterable=r.iter_content(chunk_size=1024), total=total_size / 1024, unit='KB'):
            f.write(data)

In [ ]:
def extract_shapefile(estados_geo, directory, shp_dir, shape_type):
    for estado in estados_geo:
        file = estado.split('_')[0]
        zip_file = os.path.join(directory, estado)
        shp_files = [
            f'conjunto_de_datos/{file}{shape_type}.shp',
            f'conjunto_de_datos/{file}{shape_type}.cpg',
            f'conjunto_de_datos/{file}{shape_type}.dbf',
            f'conjunto_de_datos/{file}{shape_type}.prj',
            f'conjunto_de_datos/{file}{shape_type}.shx'
        ]

        # Verificar si todos los archivos necesarios ya están descomprimidos
        if all(os.path.exists(os.path.join(shp_dir, f)) for f in shp_files):
            print(f"Todos los archivos para {file} ya están descomprimidos en {shp_dir}")
            continue

        # Proceso de descompresión
        with ZipFile(zip_file, 'r') as zip:
            for f in shp_files:
                try:
                    zip.extract(f, shp_dir)
                except KeyError:
                    if f.endswith('.cpg'):  # Archivo CPG es opcional
                        with open(os.path.join(shp_dir, f), 'w') as out_file:
                            out_file.write("ISO 88591")
                    else:
                        print(f"El archivo {f} no se encontró en el ZIP.")

In [ ]:
estados_geo = ["01_aguascalientes.zip","02_bajacalifornia.zip","03_bajacaliforniasur.zip","04_campeche.zip","05_coahuiladezaragoza.zip","06_colima.zip","07_chiapas.zip","08_chihuahua.zip","09_ciudaddemexico.zip","10_durango.zip","11_guanajuato.zip","12_guerrero.zip","13_hidalgo.zip","14_jalisco.zip","15_mexico.zip","16_michoacandeocampo.zip","17_morelos.zip","18_nayarit.zip","19_nuevoleon.zip","20_oaxaca.zip","21_puebla.zip","22_queretaro.zip","23_quintanaroo.zip","24_sanluispotosi.zip","25_sinaloa.zip","26_sonora.zip","27_tabasco.zip","28_tamaulipas.zip","29_tlaxcala.zip","30_veracruzignaciodelallave.zip","31_yucatan.zip","32_zacatecas.zip"]
estados_num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13 ,14 ,15, 16, 17, 18, 19, 20, 21, 22, 23 ,24, 25, 26, 27, 28, 29, 30, 31, 32]

In [ ]:
download_directory = "./inegi/ccpvagebmza/"
csv_directory = download_directory + "csv/"

os.makedirs(download_directory, exist_ok=True)
os.makedirs(csv_directory, exist_ok=True)
os.makedirs("./inegi/mgccpv/gpkg", exist_ok=True)


In [ ]:
# Descargar datos estadísticos por estado
for i in estados_num:
    estado = str(i).zfill(2)
    zip_file_path = f'{download_directory}ageb_mza_urbana_{estado}_cpv2020_csv.zip'
    csv_file_path = f'{csv_directory}conjunto_de_datos/conjunto_de_datos_ageb_urbana_{estado}_cpv2020.csv'

    # Comprobar si el archivo CSV ya ha sido extraído
    if not os.path.exists(csv_file_path):
        url = f'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/ageb_mza_urbana_{estado}_cpv2020_csv.zip'

        # Comprobar si el archivo ZIP ya existe antes de descargarlo
        if not os.path.exists(zip_file_path):
            download(url, download_directory)  # Asegúrate de que la función `download` acepte el directorio de destino
        # Descomprimir el archivo CSV específico si el ZIP ya está disponible
        with ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extract(f'ageb_mza_urbana_{estado}_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{estado}_cpv2020.csv', csv_directory)
    else:
        print(f'El archivo CSV para el estado {estado} ya existe y no se descargará ni descomprimirá de nuevo.')

2566KB [00:01, 2001.81KB/s]                                    
100%|█████████▉| 8603/8626.7998046875 [00:03<00:00, 2608.79KB/s]/usr/local/lib/python3.10/dist-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 8627/8626.7998046875 [00:03<00:00, 2539.89KB/s]
2321KB [00:01, 1644.06KB/s]                                     
100%|██████████| 1798/1797.6279296875 [00:00<00:00, 2335.91KB/s]
8063KB [00:03, 2519.29KB/s]                                     
2073KB [00:00, 2396.28KB/s]                                     
100%|██████████| 7931/7930.98828125 [00:03<00:00, 2507.53KB/s]
9982KB [00:03, 2549.56KB/s]                                   
12687KB [00:09, 1392.14KB/s]                                     
4203KB [00:02, 1479.58KB/s]                                    
10119KB [00:11, 844.88KB/s]                                      
7406KB [00:03, 2462.92KB/s]                                     
100%|██████████| 5741/5740.9990234375

In [ ]:
url_mgccpv = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/"
for state in estados_geo:
    download(url_mgccpv + state, "./inegi/mgccpv")

# Manzanas
shape_type = "m"
directory= "./inegi/mgccpv/"
shp_dir = directory+"shp/m/"
extract_shapefile(estados_geo,directory,shp_dir,shape_type)

35903KB [00:15, 2361.33KB/s]                                      
100%|██████████| 66733/66732.7880859375 [00:26<00:00, 2518.41KB/s]
32043KB [00:27, 1171.33KB/s]                                     
100%|██████████| 32889/32888.828125 [00:18<00:00, 1811.86KB/s]
100%|██████████| 65732/65731.93359375 [00:35<00:00, 1864.50KB/s]
100%|██████████| 28298/28297.5615234375 [00:10<00:00, 2587.03KB/s]
100%|██████████| 106237/106236.5068359375 [01:12<00:00, 1470.31KB/s]
100%|██████████| 103312/103311.6591796875 [01:13<00:00, 1414.88KB/s]
81258KB [00:31, 2589.68KB/s]                                      
60160KB [00:34, 1757.65KB/s]                                     
100%|██████████| 130036/130035.5380859375 [01:31<00:00, 1421.72KB/s]
122744KB [01:16, 1612.42KB/s]                                       
100%|██████████| 107392/107391.822265625 [00:45<00:00, 2364.57KB/s]
100%|██████████| 139284/139283.65625 [00:58<00:00, 2370.66KB/s]
100%|██████████| 221411/221410.6123046875 [02:22<00:00, 1552.42K

In [ ]:
for estado in estados_geo:
    file = estado.split('_')[0]
    print("procesando estado: "+ file)
    gpdf = gpd.read_file(f"./inegi/mgccpv/shp/m/conjunto_de_datos/{file}m.shp")
    df = pd.read_csv(f"./inegi/ccpvagebmza/csv/ageb_mza_urbana_{file}_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{file}_cpv2020.csv",na_values=['N/A','N/D','*'])
    df = df[["ENTIDAD", "MUN", "LOC", "AGEB", "MZA","POBTOT", "P_15A17", "P_18A24"]]
    df['CVEGEO'] = df.apply(lambda row: str(row['ENTIDAD']).zfill(2) + str(row['MUN']).zfill(3)+ str(row['LOC']).zfill(4)+ str(row['AGEB']).zfill(4)+ str(row['MZA']).zfill(3), axis=1)
    df_geo_censo = pd.merge(df, gpdf, how = 'left').drop(["CVE_ENT", "CVE_MUN", "CVE_LOC", "CVE_AGEB", "CVE_MZA"], axis = 1)
    df_geo_censo = df_geo_censo.drop(df[df.MZA == 0].index)
    df_geo_censo = gpd.GeoDataFrame(df_geo_censo, geometry="geometry")
    print("guardando datos del estado: "+str(estado))
    df_geo_censo.to_file(f"./inegi/mgccpv/gpkg/cpv2020_{file}.shp")

procesando estado: 01
guardando datos del estado: 01_aguascalientes.zip
procesando estado: 02
guardando datos del estado: 02_bajacalifornia.zip
procesando estado: 03
guardando datos del estado: 03_bajacaliforniasur.zip
procesando estado: 04
guardando datos del estado: 04_campeche.zip
procesando estado: 05
guardando datos del estado: 05_coahuiladezaragoza.zip
procesando estado: 06
guardando datos del estado: 06_colima.zip
procesando estado: 07
guardando datos del estado: 07_chiapas.zip
procesando estado: 08
guardando datos del estado: 08_chihuahua.zip
procesando estado: 09
guardando datos del estado: 09_ciudaddemexico.zip
procesando estado: 10
guardando datos del estado: 10_durango.zip
procesando estado: 11
guardando datos del estado: 11_guanajuato.zip
procesando estado: 12
guardando datos del estado: 12_guerrero.zip
procesando estado: 13
guardando datos del estado: 13_hidalgo.zip
procesando estado: 14
guardando datos del estado: 14_jalisco.zip
procesando estado: 15
guardando datos del 

<ipython-input-12-e99e75586401>:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"./inegi/ccpvagebmza/csv/ageb_mza_urbana_{file}_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{file}_cpv2020.csv",na_values=['N/A','N/D','*'])


guardando datos del estado: 17_morelos.zip
procesando estado: 18
guardando datos del estado: 18_nayarit.zip
procesando estado: 19
guardando datos del estado: 19_nuevoleon.zip
procesando estado: 20
guardando datos del estado: 20_oaxaca.zip
procesando estado: 21
guardando datos del estado: 21_puebla.zip
procesando estado: 22
guardando datos del estado: 22_queretaro.zip
procesando estado: 23
guardando datos del estado: 23_quintanaroo.zip
procesando estado: 24
guardando datos del estado: 24_sanluispotosi.zip
procesando estado: 25
guardando datos del estado: 25_sinaloa.zip
procesando estado: 26
guardando datos del estado: 26_sonora.zip
procesando estado: 27
guardando datos del estado: 27_tabasco.zip
procesando estado: 28
guardando datos del estado: 28_tamaulipas.zip
procesando estado: 29
guardando datos del estado: 29_tlaxcala.zip
procesando estado: 30
guardando datos del estado: 30_veracruzignaciodelallave.zip
procesando estado: 31
guardando datos del estado: 31_yucatan.zip
procesando esta

In [ ]:
dfs = []
for estado in estados_geo:
    file = estado.split('_')[0]
    gdf = gpd.read_file(f"./inegi/mgccpv/gpkg/cpv2020_{file}.shp")
    dfs.append(gdf)

censo_df = pd.concat(dfs, ignore_index=True)

In [ ]:
censo_df.loc[:, 'centroid'] = censo_df['geometry'].centroid

In [ ]:
censo_df_centroide = censo_df.set_geometry('centroid')

In [ ]:
censo_df_centroide.drop(columns=["geometry"], inplace=True)

In [ ]:
censo_df_centroide

,ENTIDAD,MUN,LOC,AGEB,MZA,POBTOT,P_15A17,P_18A24,CVEGEO,AMBITO,TIPOMZA,centroid
0,1,1,1,0017,1,170,8.0,39.0,0100100010017001,Urbana,Típica,POINT (2473563.710 1103227.586)
1,1,1,1,0017,2,198,10.0,35.0,0100100010017002,Urbana,Típica,POINT (2473598.732 1103253.809)
2,1,1,1,0017,3,198,14.0,28.0,0100100010017003,Urbana,Típica,POINT (2473607.423 1103079.709)
3,1,1,1,0017,4,202,10.0,27.0,0100100010017004,Urbana,Típica,POINT (2473524.417 1103197.511)
4,1,1,1,0017,5,157,10.0,30.0,0100100010017005,Urbana,Típica,POINT (2473444.521 1103305.702)
...,...,...,...,...,...,...,...,...,...,...,...,...
1611443,32,58,1,0123,18,2,NaN,NaN,3205800010123018,Urbana,Típica,POINT (2355652.160 1057050.164)
1611444,32,58,1,0123,19,0,0.0,0.0,3205800010123019,Urbana,Típica,POINT (2355590.847 1056996.121)
1611445,32,58,1,0123,20,0,0.0,0.0,3205800010123020,Urbana,Típica,POINT (2355160.371 1057028.549)
1611446,32,58,1,0123,21,2,NaN,NaN,3205800010123021,Urbana,Típica,POINT (2355098.445 1057130.014)


In [ ]:
censo_df_centroide.to_file(f"./inegi/mgccpv/gpkg/cpv2020_centroides_INE.shp")